In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split

import cv2

import numpy as np

from matplotlib import pyplot as plt

import os
import shutil

print(tf.__version__)
print(tf.config.list_physical_devices())

2.8.0
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
class blocc(layers.Layer):
    def __init__(self, filters, k_size, stride = 1, pad = 0, activation = layers.LeakyReLU()):
        super().__init__()
        self.pad = pad
        self.conv1 = layers.Conv2D(filters, kernel_size = k_size, strides = stride, padding = 'valid')
        self.activation = activation
        self.batchNorm = layers.BatchNormalization()

    def call(self, input_tensor, training = False):
        input = input_tensor
        if self.pad != 0:
            input = tf.pad(input, [[0, 0], [self.pad, self.pad], [self.pad, self.pad], [0, 0]], 'constant')

        x = self.conv1(input)
        x = self.batchNorm(x, training = training)
        return self.activation(x)

class simplebloc(layers.Layer):
    def __init__(self, out1, out3, activation = layers.LeakyReLU()):
        super().__init__()

        self.conv1 = blocc(out1, 1, pad = 0, activation = activation)
        self.conv2 = blocc(out3, 3, pad = 1, activation = activation)

    def call(self, input):
        conv1_out = self.conv1(input)
        conv2_out = self.conv2(input)

        output = tf.concat(values = [conv1_out, conv2_out], axis = 3)
        return output

class full_model(keras.Model):
    def __init__(self, in_channels = 3, out_channels = 1):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.activation = layers.LeakyReLU(0.01)
        self.final_activation = layers.LeakyReLU(0.01)
        self.patch_size = 32

        self.conv1 = blocc(64, 3, pad = self.patch_size, activation = self.activation)
        self.simple1 = simplebloc(16, 16, activation = self.activation)
        self.simple2 = simplebloc(16, 32, activation = self.activation)
        self.conv2 = blocc(16, 14, activation = self.activation)
        self.simple3 = simplebloc(112, 48, activation=self.activation)
        self.simple4 = simplebloc(64, 32, activation=self.activation)
        self.simple5 = simplebloc(40, 40, activation=self.activation)
        self.simple6 = simplebloc(32, 96, activation=self.activation)
        self.conv3 = blocc(32, 18, activation=self.activation)
        self.conv4 = blocc(64, 1, activation=self.activation)
        self.conv5 = blocc(64, 1, activation=self.activation)

        self.conv6 = blocc(self.out_channels, 1, activation = self.final_activation)

    def call(self, input):
        x = self.conv1(input)
        x = self.simple1(x)
        x = self.simple2(x)
        x = self.conv2(x)
        x = self.simple3(x)
        x = self.simple4(x)
        x = self.simple5(x)
        x = self.simple6(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        return x

In [3]:
def load_data():
    notation_path = '../mbm_data/labels'
    image_path = '../mbm_data/image'
    notations = []
    images = []

    dot_list = os.listdir(notation_path)
    image_list = os.listdir(image_path)

    for entry in dot_list:
        img = cv2.imread(os.path.join(notation_path, entry))
        # Insert any image preprocessing here!
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = np.float32(img)
        img = cv2.normalize(img, None, 0, 1, cv2.NORM_MINMAX)
        img = img[..., np.newaxis]
        
        
        notations.append(img)

    for entry in image_list:
        img = cv2.imread(os.path.join(image_path, entry))

        # Insert any image preprocessing here! TODO

        images.append(img)

    return np.array(images), np.array(notations)

def make_labels(debug = False):
    file_path = '../mbm_data'
    dir_name = 'labels'
    dot_path = '../mbm_data/notation'

    cur_dir = os.getcwd()
    label_maker = layers.Conv2D(1, 32, padding='same', use_bias=False, kernel_initializer='Ones')
    ret = []
    notations = []

    dot_list = os.listdir(dot_path)

    for entry in dot_list:
        img = cv2.imread(os.path.join(dot_path, entry))

        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.normalize(img, None, 0, 1, cv2.NORM_MINMAX)
        img = np.float32(img)
        img = img[..., np.newaxis]
        notations.append(img)

    notations = np.array(notations)
    pads = tf.pad(notations, [[0, 0], [16, 16], [16, 16], [0, 0]], 'constant')
    heat = label_maker(pads)
    heat = heat.numpy()

    if debug:
        return heat

    direct_path = os.path.join(file_path, dir_name)
    if os.path.exists(direct_path):
        shutil.rmtree(direct_path)

    os.mkdir(direct_path)
    
    os.chdir(direct_path)
    for i, img in enumerate(heat):
        img_save = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)
        cv2.imwrite('map' + dot_list[i], img_save)

    os.chdir(cur_dir)

In [4]:
#h = make_labels()

In [5]:
random_seed = 56
x, y = load_data()

In [6]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.15, random_state=random_seed
)

x_train, x_val, y_train, y_val = train_test_split(
    x_train, y_train, test_size=0.15, random_state=random_seed
)

x_train = np.float32(x_train)

In [7]:
print('Training: ', x_train.shape)
print('Test: ', x_test.shape)
print('Validation: ', x_val.shape)

print('-----Labels-----')
print('Training: ', y_train.shape)
print('Test: ', y_test.shape)
print('Validation: ', y_val.shape)

Training:  (31, 600, 600, 3)
Test:  (7, 600, 600, 3)
Validation:  (6, 600, 600, 3)
-----Labels-----
Training:  (31, 632, 632, 1)
Test:  (7, 632, 632, 1)
Validation:  (6, 632, 632, 1)


### Loss: MAE/MSE
### Optimizer: Adam
### Epochs: 15
### Batches: TBD

In [8]:
print(tf.test.is_built_with_cuda())
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

True
Num GPUs Available:  1


In [9]:
lr = 0.001
epochs = 15
batch = 1

#loss = keras.losses.mean_absolute_error()
opt = keras.optimizers.Adam(learning_rate=lr)

model = full_model()

model.compile(optimizer=opt, loss='mean_absolute_error', metrics=['Accuracy'])

In [10]:
shape = x_test.shape
model.build(input_shape=shape)
model.summary()

Model: "full_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 leaky_re_lu_2 (LeakyReLU)   multiple                  0         
                                                                 
 leaky_re_lu_3 (LeakyReLU)   multiple                  0         
                                                                 
 blocc (blocc)               multiple                  2048      
                                                                 
 simplebloc (simplebloc)     multiple                  10400     
                                                                 
 simplebloc_1 (simplebloc)   multiple                  9968      
                                                                 
 blocc_5 (blocc)             multiple                  150608    
                                                                 
 simplebloc_2 (simplebloc)   multiple                  9

In [11]:
model.fit(x_train, y_train, epochs=epochs, batch_size=batch, validation_data=(x_val, y_val))

Epoch 1/15


ResourceExhaustedError: Graph execution error:

Detected at node 'full_model/simplebloc_2/blocc_6/conv2d_6/Conv2D' defined at (most recent call last):
    File "C:\Users\asus\anaconda3\envs\DeepLearning_TensorFlow\lib\runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\asus\anaconda3\envs\DeepLearning_TensorFlow\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\asus\anaconda3\envs\DeepLearning_TensorFlow\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "C:\Users\asus\anaconda3\envs\DeepLearning_TensorFlow\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\asus\anaconda3\envs\DeepLearning_TensorFlow\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "C:\Users\asus\anaconda3\envs\DeepLearning_TensorFlow\lib\site-packages\tornad